In [ ]:
import logging
import boto3
import boto3.session
from datetime import date, datetime
from botocore.exceptions import ClientError
import json
import sys
import time
import os
import argparse
import aws_vpc_destroy as destroy
import pandas as pd

# http://boto3.readthedocs.io/en/latest/reference/services/ec2.html#service-resource

ec2 = boto3.resource('ec2')
client = boto3.client('ec2')

In [ ]:
# session = boto3.session.Session()
REGION = "us-east-1"
cidr_block = '10.0.0.0/16'

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

In [ ]:
# create VPC
vpc = ec2.create_vpc(CidrBlock=cidr_block)
# we can assign a name to vpc, or any resource, by using tag
vpc.create_tags(Tags=[{"Key": "Name", "Value": "python-vpc"}])
vpc.wait_until_available()
print(vpc.id)

In [ ]:
# Code below also creates a vpc (alternative to the above)
# ec2_client = session.client('ec2', REGION)
# ec2_resource = session.resource('ec2', REGION)
# create_vpc_response = ec2_client.create_vpc(CidrBlock=cidr_block)
# vpc = ec2_resource.Vpc(create_vpc_response["Vpc"]["VpcId"])

In [ ]:
# create then attach internet gateway
igw = ec2.create_internet_gateway()
igw.create_tags(Tags=[{"Key": "Name", "Value": "python-vpc-igw"}])
#attach to vpc using the gateway id
vpc.attach_internet_gateway(InternetGatewayId=igw.id)
print(igw.id)

#create_internet_gateway also returns a dict
# create_ig_response = ec2_client.create_internet_gateway()
# ig_id = create_ig_response["InternetGateway"]["InternetGatewayId"]

## Create public and private subnets

In [ ]:
cidr_blocks = ["10.0.1.0/24", "10.0.2.0/24", "10.0.3.0/24", "10.0.4.0/24"]
availability_zones = ["us-east-1a", "us-east-1b"]

# create subnet
#define a function to create subnets
def create_public_private_subnets(cidr_list, azs, vpc_id):
    """input: cidr_block list, availability zones' list and vpc_id
    output: list of public and private subnets"""
    #initialize empty list for subnets
    subnets = []
    for idx in range(len(cidr_list)):
        #because I want a public and private SN to be in same az
        if idx % 2 == 0:
            subnet = ec2.create_subnet(CidrBlock=cidr_list[idx], VpcId=vpc_id, \
                AvailabilityZone = azs[0])
            subnets.append(subnet)
        elif idx % 2 == 1:
            subnet = ec2.create_subnet(CidrBlock=cidr_list[idx], VpcId=vpc_id, \
                AvailabilityZone = azs[1])            
            subnets.append(subnet)
    #create tags
    for idx in range(len(subnets)):
        if idx % 2 == 0:
            subnets[idx].create_tags(Tags=[{"Key": "Name", "Value": "publicSN_" + str(idx+1)}])
        else:
            subnets[idx].create_tags(Tags=[{"Key": "Name", "Value": "privateSN_" + str(idx)}])

    return subnets


#call subnet function and loop print subnet ids
subnets = create_public_private_subnets(cidr_blocks, availability_zones, vpc.id)

for subnet in subnets:
    print(subnet.id)

#subnet = ec2.create_subnet(CidrBlock='192.168.1.0/24', VpcId=vpc.id)
#or 
#create_subnet can be run directly on the vpc object
# subnet = vpc.create_subnet(CidrBlock=subnet_cidr, AvailabilityZone="{}{}".format(REGION, az))

In [ ]:
def wait_nat_creation(nat_gateway_id):
    """
    Check if successful state is reached every 15 seconds until a successful state is reached.
    An error is returned after 40 failed checks.
    """
    try:
        waiter = client.get_waiter('nat_gateway_available')
        waiter.wait(NatGatewayIds=[nat_gateway_id])
    except ClientError:
        logger.exception(f'Could not create the NAT gateway.')
        raise

In [ ]:
#create NAT gateway 
#initialize elastic ip for same
allocation = client.allocate_address(Domain='vpc')
# response = ec2.associate_address(AllocationId=allocation['AllocationId'],
#                                     InstanceId='INSTANCE_ID')
# print(response)
nat_gw = client.create_nat_gateway(SubnetId=subnets[0].id, AllocationId=allocation['AllocationId'],
            TagSpecifications=[{
                'ResourceType':
                'natgateway',
                'Tags': [{
                    'Key': 'Name',
                    'Value': 'python-nat-gateway'
                }]
            }])

nat_gw_id = nat_gw['NatGateway']['NatGatewayId']

#wait until nat gateway is created
wait_nat_creation(nat_gw_id)

In [ ]:
print(nat_gw_id)

## Create route tables and associations

In [ ]:
# # create a public route table, this can be used when creating a single route table

# public_route_table = vpc.create_route_table()
# # and a public route, assign public cidr block and attach to internet gateway created above
# route = public_route_table.create_route(
#     DestinationCidrBlock='0.0.0.0/0',
#     GatewayId=ig.id
# )
# # associate the route table with the subnet
# public_route_table.associate_with_subnet(SubnetId=subnet.id)

# print(public_route_table.id)

## Code below for multiple route tables


In [ ]:
# define a function to create more than one route table at a time

def route_tables(rt_titles, gw_list):
    """input: cidr_block list, availability zones' list and vpc_id
    output: list of public and private subnets"""
    #initialize empty route table list to collect route_tables
    route_tables = []
    for idx in range(len(rt_titles)):

        title = vpc.create_route_table()

        # and a public route, assign public cidr block and attach to internet gateway created above
        g_way = gw_list[idx]
        if type(g_way) is dict:

            route = title.create_route(
                DestinationCidrBlock='0.0.0.0/0',
                GatewayId=g_way['NatGateway']['NatGatewayId']
            )
        # associate the route table with the subnet
        # for idx2 in range(len(rt_titles)):
            title.associate_with_subnet(SubnetId=subnets[idx].id)
            idx2 = idx + 2
            title.associate_with_subnet(SubnetId=subnets[idx2].id)

            route_tables.append(title)
        else:

            route = title.create_route(
                DestinationCidrBlock='0.0.0.0/0',
                GatewayId=g_way.id
            )
        # associate the route table with the subnet
        # for idx2 in range(len(rt_titles)):
            title.associate_with_subnet(SubnetId=subnets[idx].id)
            idx2 = idx + 2
            title.associate_with_subnet(SubnetId=subnets[idx2].id)

            route_tables.append(title)

    #iterate to create tags for route tables
    for idx in range(len(route_tables)):
        route_tables[idx].create_tags(Tags=[{"Key": "Name", "Value": str(rt_titles[idx])}])

    return route_tables


In [ ]:
#initialize route table variables
rt_titles = ["public_route_table", "private_route_table"]
gws = [igw, nat_gw]

#call route table function to create public and private route tables
rt_tables = route_tables(rt_titles, gws)

print(rt_tables)

## Create frontend and backend security groups

In [ ]:
# Create frontend sec group
frontend_sg = ec2.create_security_group(
    GroupName='frontend_sg_py_vpc', Description='front end security group', VpcId=vpc.id)

In [ ]:
#attach front end security permissions
frontend_sg.authorize_ingress(IpPermissions = [
    {
    'IpProtocol':'tcp',
    'FromPort':443,
    'ToPort':443,
    'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
    },
    {
    'IpProtocol':'tcp',
    'FromPort':80,
    'ToPort':80,
    'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
    },
    {
    'IpProtocol':'tcp',
    'FromPort':22,
    'ToPort':22,
    'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
    },
]
)

# frontend_sg.authorize_egress(IpPermissions = [
#     {
#     'IpProtocol':'-1',
#     'FromPort':0,
#     'ToPort':0,
#     'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
#     }
# ]
# )



In [ ]:
#Create backend security group
backend_sg = ec2.create_security_group(
    GroupName='backend_sg_py_vpc', Description='back end security group', VpcId=vpc.id)

In [ ]:
##attach back end security permissions

backend_sg.authorize_ingress(IpPermissions = [    
    {
    'IpRanges':[{'CidrIp':"10.0.1.0/24", 'CidrIp':"10.0.2.0/24"}],
    'IpProtocol':'tcp',
    'FromPort':3306,
    'ToPort':3306
    },
    {
    'IpRanges':[{'CidrIp':"10.0.1.0/24", 'CidrIp':"10.0.2.0/24"}],
    'IpProtocol':'tcp',
    'FromPort':22,
    'ToPort':22
    }
]
)
# egress already created with sec group
# backend_sg.authorize_egress(IpPermissions = [
#     {    
#     'IpRanges':[{'CidrIp':'0.0.0.0/0'}],
#     'IpProtocol':'-1',
#     'FromPort':0,
#     'ToPort':0
#     }
# ]
# )


In [ ]:
print(frontend_sg.id, backend_sg.id)

## Create S3 bucket and policy

In [ ]:
# Create a bucket policy
bucket_name = 'py-med-bucket'

# s3_location = {
#     'LocationConstraint':'us-east-1'
# }
# , CreateBucketConfiguration=s3_location
bucket_policy = {
    'Version': '2012-10-17',
    'Statement': [{
        'Sid': 'PublicReadGetObject',
        'Effect': 'Allow',
        'Principal': '*',
        'Action': ['s3:GetObject', 's3:GetObjectVersion'],
        'Resource': f'arn:aws:s3:::{bucket_name}/*'
    }]
}

# Convert the policy from JSON dict to string
bucket_policy = json.dumps(bucket_policy)

# Set the new policy and include ACL
s3 = boto3.client('s3')
s3.create_bucket(Bucket=bucket_name, ACL='public-read')
s3.put_bucket_policy(Bucket=bucket_name, Policy=bucket_policy)

## Create Apache web server instance with user data

In [ ]:
#use code to creat key if you do not have one
# ec2_key = boto3.connect_ec2()
# key = ec2_key = boto3.connect_ec2().create_key_pair('mynewkey')
# key.save('~/filepath of your choice/')

## Initialize Apache and SQL command line scripts

In [ ]:
APACHE_SCRIPT = """#!/bin/bash
  #sudo yum update -y
  sudo yum install httpd -y
  sudo service httpd start
  sudo chkconfig httpd on
  cd /var/www/html
  echo "<html><h1>This is Apache Web Server 01</h1></html>" > index.html
  sudo yum install mysql -y
  """

SQLDB_SCRIPT = """#!/bin/bash
sudo yum update -y
sudo yum install mysql-server -y
sudo service mysqld start
"""

## Create Apache web server

In [ ]:
# Create instance
apache_server = ec2.create_instances(
    ImageId='ami-0b0af3577fe5e3532', InstanceType='t2.micro', MaxCount=1, MinCount=1, KeyName='win_keypair',
    NetworkInterfaces=[{'SubnetId': subnets[0].id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [frontend_sg.group_id]}],
    UserData=APACHE_SCRIPT)

#wait until apache is running to provide output
apache_server[0].wait_until_running()

print(apache_server[0].id)

## Create MySQL database server

In [ ]:
# Create MySQL instance and assign to back end security group
sql_server = ec2.create_instances(
    ImageId='ami-0b0af3577fe5e3532', InstanceType='t2.micro', MaxCount=1, MinCount=1, KeyName='win_keypair',
    NetworkInterfaces=[{'SubnetId': subnets[1].id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [backend_sg.group_id]}],
    UserData=SQLDB_SCRIPT)

#wait until apache is running to provide output
sql_server[0].wait_until_running()

print(sql_server[0].id)

## Print infrastructure summary dataframe

## Destroy EC2 instances and VPC

In [ ]:
#call vpc destroy function and insert vpcid and region as inputs
destroy.destroy_ec2(str(vpc.id), REGION)

In [ ]:
#call vpc destroy function and insert vpcid and region as inputs
destroy.delete_vpc(str(vpc.id), REGION, release_eips=True)